In [2]:
import numpy as np
import pandas as pd

import sys
import os

sys.path.append(os.path.abspath('..'))
from utils.db import get_db

In [3]:
engine = get_db()

In [20]:
query = """select customer_id, storeid, sale_date, sale_id, total_sales, qty, member_level
from sales
where store_number='440'
and sale_date between '5/1/2025' and '6/7/2025'
and sale_type = 'Sale'
and item_ring_type = 'ITEM'
and customer_id is not null
order by sale_date, sale_id, customer_id
"""

In [21]:
ds = pd.read_sql(query, engine)

2025-06-13 23:14:51,884 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-13 23:14:51,884 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-06-13 23:14:51,885 INFO sqlalchemy.engine.Engine [cached since 1445s ago] {'table_name': "select customer_id, storeid, sale_date, sale_id, total_sales, qty, member_level\nfrom sales\nwhere store_number='440'\nand sale_date between '5/1/2025' and '6/7/2025'\nand sale_type = 'Sale'\nand item_ring_type = 'ITEM'\nand customer_id is not null\norder by sale_date, sale_id, customer_id\n", 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'para

In [23]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126080 entries, 0 to 126079
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   customer_id   126080 non-null  object        
 1   storeid       126080 non-null  int64         
 2   sale_date     126080 non-null  datetime64[ns]
 3   sale_id       126080 non-null  int64         
 4   total_sales   126080 non-null  float64       
 5   qty           126080 non-null  float64       
 6   member_level  126080 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 6.7+ MB


In [22]:
ds.head()

,customer_id,storeid,sale_date,sale_id,total_sales,qty,member_level
0,40000005377.0,1397,2025-05-01,102084,39.89,1.0,8
1,40000007146.0,1397,2025-05-01,102091,48.89,1.0,8
2,40000005605.0,1397,2025-05-01,102093,42.89,1.0,8
3,40000006293.0,1397,2025-05-01,102094,14.49,1.0,8
4,40000005524.0,1397,2025-05-01,102095,40.00,1.0,8


In [24]:
ds['sale_date'] = pd.to_datetime(ds['sale_date'])

In [25]:
ds['customer_id'] = ds['customer_id'].astype(str).str.replace('.0', '', regex=False)

In [26]:
ds.head()

,customer_id,storeid,sale_date,sale_id,total_sales,qty,member_level
0,40000005377,1397,2025-05-01,102084,39.89,1.0,8
1,40000007146,1397,2025-05-01,102091,48.89,1.0,8
2,40000005605,1397,2025-05-01,102093,42.89,1.0,8
3,40000006293,1397,2025-05-01,102094,14.49,1.0,8
4,40000005524,1397,2025-05-01,102095,40.00,1.0,8


In [27]:
grouped = ds.groupby(['customer_id', 'sale_id', 'sale_date']).agg({
    'total_sales': 'sum',
    'qty': 'sum',
    'storeid': 'first',
    'member_level': 'first'
}).reset_index()

In [28]:
grouped

,customer_id,sale_id,sale_date,total_sales,qty,storeid,member_level
0,40000000170,254043,2025-05-25,49.43,18.0,1397,0
1,40000000170,254044,2025-05-25,12.47,3.0,1397,0
2,40000000170,375721,2025-06-01,23.23,6.0,1397,0
3,40000000170,477933,2025-05-08,66.56,17.0,1397,0
4,40000000170,478018,2025-05-09,104.37,38.0,1397,0
...,...,...,...,...,...,...,...
13888,440860,476930,2025-05-04,17.98,1.0,1397,0
13889,440860,478553,2025-05-11,17.98,1.0,1397,0
13890,440860,480156,2025-05-18,17.98,1.0,1397,0
13891,440860,481756,2025-05-25,17.98,1.0,1397,0


In [29]:
grouped.sort_values(by=['customer_id', 'sale_date'], inplace=True)

In [30]:
grouped

,customer_id,sale_id,sale_date,total_sales,qty,storeid,member_level
3,40000000170,477933,2025-05-08,66.56,17.0,1397,0
4,40000000170,478018,2025-05-09,104.37,38.0,1397,0
0,40000000170,254043,2025-05-25,49.43,18.0,1397,0
1,40000000170,254044,2025-05-25,12.47,3.0,1397,0
2,40000000170,375721,2025-06-01,23.23,6.0,1397,0
...,...,...,...,...,...,...,...
13888,440860,476930,2025-05-04,17.98,1.0,1397,0
13889,440860,478553,2025-05-11,17.98,1.0,1397,0
13890,440860,480156,2025-05-18,17.98,1.0,1397,0
13891,440860,481756,2025-05-25,17.98,1.0,1397,0


In [31]:
grouped['days_since_last'] = grouped.groupby('customer_id')['sale_date'].diff().dt.days

In [33]:
grouped.sort_values(by='days_since_last', ascending=False)


,customer_id,sale_id,sale_date,total_sales,qty,storeid,member_level,days_since_last
7946,41440000380,103588,2025-06-07,0.00,0.0,1397,8,37.0
11212,41440010854,484490,2025-06-06,22.33,6.0,1397,0,35.0
9966,41440006999,127006,2025-06-06,43.37,11.0,1397,0,35.0
6932,40000007398,153256,2025-06-07,19.18,7.0,1397,0,34.0
13594,44000348738,177007,2025-06-02,16.98,5.0,1397,0,32.0
...,...,...,...,...,...,...,...,...
13866,440802,478038,2025-05-09,55.62,3.0,1397,0,NaN
13870,440804,476193,2025-05-01,10.09,1.0,1397,0,NaN
13876,440809,476921,2025-05-04,86.93,2.0,1397,0,NaN
13887,440813,477621,2025-05-07,19.99,1.0,1397,0,NaN


In [35]:
grouped['last_week'] = grouped['days_since_last'] <= 7

In [36]:
grouped.sort_values(by=['last_week'], ascending=False)

,customer_id,sale_id,sale_date,total_sales,qty,storeid,member_level,days_since_last,last_week
6941,40000007426,126817,2025-06-04,2.79,2.0,1397,0,4.0,True
8041,41440000586,103012,2025-05-24,40.00,1.0,1397,4,1.0,True
8046,41440000586,107611,2025-05-26,43.28,15.0,1397,0,1.0,True
8054,41440000586,176189,2025-05-28,10.92,6.0,1397,0,2.0,True
8047,41440000586,107860,2025-05-29,34.55,11.0,1397,0,1.0,True
...,...,...,...,...,...,...,...,...,...
9161,41440005412,174384,2025-05-17,20.36,4.0,1397,0,15.0,False
9160,41440005412,147749,2025-05-02,33.24,10.0,1397,0,NaN,False
2826,40000005697,102952,2025-05-22,49.64,1.0,1397,8,14.0,False
2840,40000005699,476709,2025-05-03,81.04,19.0,1397,0,NaN,False
